In [4]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def create_schedule_matrix():
    """Crea una matriz de horario vacía (5 días x 9 bloques)."""
    return [[False for _ in range(9)] for _ in range(5)]

def analyze_room_schedule(room_data):
    """Analiza el horario de una sala específica."""
    schedule = create_schedule_matrix()
    total_blocks = 45  # 9 bloques x 5 días
    
    # Verificar que tenemos los datos necesarios
    if not isinstance(room_data, dict) or 'Asignaturas' not in room_data:
        print(f"Error: Datos de sala inválidos: {room_data}")
        return None
        
    # Marcar bloques ocupados
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    for class_data in room_data.get('Asignaturas', []):
        day_idx = days[class_data['Dia']]
        block_idx = class_data['Bloque'] - 1
        if 0 <= day_idx < 5 and 0 <= block_idx < 9:
            schedule[day_idx][block_idx] = True
    
    # Calcular estadísticas
    total_occupied = sum(row.count(True) for row in schedule)
    total_unoccupied = total_blocks - total_occupied
    
    return {
        'schedule': schedule,
        'total_occupied': total_occupied,
        'total_unoccupied': total_unoccupied
    }

def create_summary_dataframe(horarios_salas):
    """Crea un DataFrame con el resumen de ocupación de todas las salas."""
    summary_data = []
    
    for sala in horarios_salas:
        if not isinstance(sala, dict) or 'Codigo' not in sala:
            print(f"Error: Estructura de sala inválida: {sala}")
            continue
            
        stats = analyze_room_schedule(sala)
        if stats is None:
            continue
            
        summary_data.append({
            'Codigo': sala['Codigo'],
            'Bloques Ocupados': stats['total_occupied'],
            'Bloques Desocupados': stats['total_unoccupied'],
            'Tasa de Ocupacion': round(stats['total_occupied'] / (stats['total_occupied'] + stats['total_unoccupied']) * 100, 2)
        })
    
    df_summary = pd.DataFrame(summary_data)
    df_summary.set_index('Codigo', inplace=True)
    
    # Guardar tabla en CSV
    df_summary.to_csv('resumen_ocupacion.csv')
    
    return df_summary

def general_occupancy(df_stats):
    """Crea un gráfico de torta mostrando la tasa de ocupación general."""
    try:
        plt.figure(figsize=(10, 6))
        
        # Calcular totales
        total_ocupados = df_stats['Bloques Ocupados'].sum()
        total_desocupados = df_stats['Bloques Desocupados'].sum()
        total = total_ocupados + total_desocupados
        
        # Calcular porcentajes
        porc_ocupados = (total_ocupados / total) * 100
        porc_desocupados = (total_desocupados / total) * 100
        
        # Crear gráfico de torta
        valores = [porc_ocupados, porc_desocupados]
        etiquetas = ['Ocupados', 'Desocupados']
        colores = ['#00629B', '#A9A9A9']  # Azul y gris
        
        patches, texts, autotexts = plt.pie(valores, 
                                          labels=etiquetas,
                                          colors=colores,
                                          autopct='%1.1f%%',
                                          startangle=90)
        
        # Ajustar color de texto
        for autotext in autotexts:
            if autotext.get_text().strip('%') == f'{porc_ocupados:.1f}':
                autotext.set_color('white')
            else:
                autotext.set_color('black')
        
        plt.title('Distribución General de Ocupación')
        plt.axis('equal')
        
        plt.savefig('ocupacion_general.png')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación general: {str(e)}")
        return False

def room_occupancy(df_stats):
    """Crea un gráfico de barras horizontal compuesto por sala."""
    try:
        # Ordenar salas por ocupación total descendente
        df_sorted = df_stats.sort_values('Bloques Ocupados', ascending=True)
        
        # Preparar datos
        salas = df_sorted.index
        ocupados = df_sorted['Bloques Ocupados']
        desocupados = df_sorted['Bloques Desocupados']
        y_pos = np.arange(len(salas))
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Barra de ocupados (azul)
        bars1 = ax.barh(y_pos, ocupados, color='#00629B', label='Ocupados')
        
        # Barra de desocupados (gris)
        bars2 = ax.barh(y_pos, desocupados, left=ocupados, 
                       color='#A9A9A9', label='Desocupados')
        
        # Configurar aspecto
        ax.set_yticks(y_pos)
        ax.set_yticklabels(salas)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Añadir etiquetas
        for bars, color_text in [(bars1, 'white'), (bars2, 'black')]:
            for bar in bars:
                width = bar.get_width()
                if width > 0:  # Solo mostrar etiqueta si hay valor
                    x = bar.get_x() + width/2
                    y = bar.get_y() + bar.get_height()/2
                    ax.text(x, y, f'{int(width)}', 
                           ha='center', va='center',
                           color=color_text)
        
        plt.title('Ocupación por Sala')
        plt.xlabel('Número de Bloques')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
        plt.tight_layout()
        plt.savefig('ocupacion_salas.png', bbox_inches='tight')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación por sala: {str(e)}")
        return False

def day_occupancy(horarios_salas):
    """Crea un gráfico de barras vertical por día de la semana."""
    try:
        # Inicializar conteos por día
        dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
        ocupados_por_dia = {dia: 0 for dia in dias}
        total_por_dia = {dia: 0 for dia in dias}
        
        # Contar bloques ocupados por día
        for sala in horarios_salas:
            for asignatura in sala['Asignaturas']:
                dia = asignatura['Dia']
                ocupados_por_dia[dia] += 1
        
        # Calcular total de bloques posibles por día
        num_salas = len(horarios_salas)
        bloques_por_dia = 9  # 9 bloques por día
        for dia in dias:
            total_por_dia[dia] = num_salas * bloques_por_dia
        
        # Preparar datos para el gráfico
        ocupados = [ocupados_por_dia[dia] for dia in dias]
        desocupados = [total_por_dia[dia] - ocupados_por_dia[dia] for dia in dias]
        
        # Crear gráfico
        fig, ax = plt.subplots(figsize=(10, 6))
        x = np.arange(len(dias))
        width = 0.35
        
        # Barras ocupados (azul)
        bars1 = ax.bar(x, ocupados, width, label='Ocupados', color='#00629B')
        
        # Barras desocupados (gris)
        bars2 = ax.bar(x, desocupados, width, bottom=ocupados,
                      label='Desocupados', color='#A9A9A9')
        
        # Configurar aspecto
        ax.set_xticks(x)
        ax.set_xticklabels(dias)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Añadir etiquetas
        for bars, color_text in [(bars1, 'white'), (bars2, 'black')]:
            for bar in bars:
                height = bar.get_height()
                if height > 0:  # Solo mostrar etiqueta si hay valor
                    ax.text(bar.get_x() + bar.get_width()/2, 
                           bar.get_y() + height/2,
                           f'{int(height)}',
                           ha='center', va='center',
                           color=color_text)
        
        plt.title('Ocupación por Día')
        plt.ylabel('Número de Bloques')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('ocupacion_dias.png')
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"Error al crear el gráfico de ocupación por día: {str(e)}")
        return False

def save_results(df_stats, total_counts, daily_stats):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_stats.to_dict(orient='index'),
            'global_stats': {
                'total_ocupados': int(total_counts[0]),
                'total_desocupados': int(total_counts[1]),
                'tasa_ocupacion_global': float(total_counts[0] / sum(total_counts) * 100)
            },
            'daily_stats': daily_stats
        }
        
        with open('metricas_ocupacion.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def calculate_daily_stats(horarios_salas):
    """Calcula estadísticas diarias de ocupación."""
    dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
    ocupados_por_dia = {dia: 0 for dia in dias}
    total_por_dia = {dia: 0 for dia in dias}
    
    num_salas = len(horarios_salas)
    bloques_por_dia = 9
    
    # Contar bloques ocupados por día
    for sala in horarios_salas:
        for asignatura in sala['Asignaturas']:
            dia = asignatura['Dia']
            ocupados_por_dia[dia] += 1
    
    # Calcular total y tasa de ocupación por día
    daily_stats = {}
    for dia in dias:
        total = num_salas * bloques_por_dia
        ocupados = ocupados_por_dia[dia]
        daily_stats[dia] = {
            'bloques_ocupados': ocupados,
            'bloques_desocupados': total - ocupados,
            'tasa_ocupacion': round(ocupados / total * 100, 2)
        }
    
    return daily_stats

def analyze_occupancy_quality(horarios_salas, salas_input):
    """
    Analiza la calidad de ocupación de cada sala considerando su capacidad real.
    """
    # Crear diccionario de capacidades
    capacidades = {sala['Codigo']: sala['Capacidad'] for sala in salas_input}
    
    quality_stats = {}
    
    for sala in horarios_salas:
        codigo = sala['Codigo']
        capacidad = capacidades[codigo]
        
        # Inicializar matriz de ocupación (5 días × 9 bloques)
        occupancy_matrix = [[0.0 for _ in range(9)] for _ in range(5)]
        
        # Mapeo de días a índices
        dias = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
        
        # Llenar matriz con porcentajes de ocupación
        for asignatura in sala['Asignaturas']:
            day_idx = dias[asignatura['Dia']]
            block_idx = asignatura['Bloque'] - 1
            
            # Capacidad es un porcentaje (1.0 = 100% de la capacidad de la sala)
            occupancy_matrix[day_idx][block_idx] = asignatura['Capacidad']
        
        # Calcular estadísticas
        total_blocks = 45  # 9 bloques × 5 días
        used_blocks = len(sala['Asignaturas'])
        
        # Clasificar bloques por nivel de ocupación
        low_quality = 0    # < 33% de la capacidad
        medium_quality = 0 # 33-66% de la capacidad
        high_quality = 0   # > 66% de la capacidad
        
        for asignatura in sala['Asignaturas']:
            usage = asignatura['Capacidad'] * 100  # Convertir a porcentaje
            if usage < 33:
                low_quality += 1
            elif usage < 66:
                medium_quality += 1
            else:
                high_quality += 1
        
        # Calcular promedio de ocupación solo en bloques usados
        if used_blocks > 0:
            # El campo Capacidad ya es un porcentaje (1.0 = 100%)
            avg_occupancy = sum(asignatura['Capacidad'] for asignatura in sala['Asignaturas']) / used_blocks
        else:
            avg_occupancy = 0
            
        # Convertir a porcentaje para consistencia con otras métricas
        avg_occupancy = avg_occupancy * 100
        
        quality_stats[codigo] = {
            'capacidad': capacidad,
            'bloques_totales': total_blocks,
            'bloques_usados': used_blocks,
            'ocupacion_baja': low_quality,
            'ocupacion_media': medium_quality,
            'ocupacion_alta': high_quality,
            'promedio_ocupacion': avg_occupancy,
            'tasa_ocupacion': (used_blocks / total_blocks) * 100,
        }
    
    return quality_stats

def create_quality_chart(quality_stats):
    """
    Crea un gráfico de barras apiladas mostrando la calidad de ocupación por sala.
    """
    # Preparar datos para el gráfico
    df = pd.DataFrame.from_dict(quality_stats, orient='index')
    
    # Calcular porcentajes relativos al total de bloques
    df['pct_baja'] = (df['ocupacion_baja'] / df['bloques_totales']) * 100
    df['pct_media'] = (df['ocupacion_media'] / df['bloques_totales']) * 100
    df['pct_alta'] = (df['ocupacion_alta'] / df['bloques_totales']) * 100
    
    # Ordenar por promedio de ocupación
    df_sorted = df.sort_values('promedio_ocupacion', ascending=True)
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(15, 10))
    
    # Posiciones de las barras
    y_pos = np.arange(len(df_sorted))
    
    # Crear barras apiladas
    bars1 = ax.barh(y_pos, df_sorted['pct_baja'], color='#ff9999', label='Baja (<33%)')
    bars2 = ax.barh(y_pos, df_sorted['pct_media'], left=df_sorted['pct_baja'], 
                   color='#99cc99', label='Media (33-66%)')
    bars3 = ax.barh(y_pos, df_sorted['pct_alta'], 
                   left=df_sorted['pct_baja'] + df_sorted['pct_media'],
                   color='#00629B', label='Alta (>66%)')
    
    # Configurar aspecto
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f"{idx} (cap:{row['capacidad']})" for idx, row in df_sorted.iterrows()])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Añadir etiquetas con el promedio de ocupación
    for i, row in enumerate(df_sorted.itertuples()):
        ax.text(100.5, i, f'Ocupación promedio de asientos: {row.promedio_ocupacion:.1f}%',
                va='center', ha='left', fontsize=8)
    
    plt.title('Calidad de Ocupación por Sala', fontsize=14, pad=20)
    plt.xlabel('Porcentaje de Bloques (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Ajustar diseño y guardar
    plt.tight_layout()
    plt.savefig('calidad_ocupacion.png', bbox_inches='tight', dpi=300)
    plt.close()

def save_quality_results(quality_stats):
    """Guarda los resultados del análisis de calidad en un archivo JSON."""
    try:
        with open('metricas_calidad_ocupacion.json', 'w', encoding='utf-8') as f:
            json.dump(quality_stats, f, ensure_ascii=False, indent=2)
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    horarios_salas = load_json_file('../../agent_output/Horarios_salas.json')
    salas_input = load_json_file('../../agent_input/InputOfSala.json')

    if not horarios_salas or not salas_input:
        return
    
    # 2. Realizar análisis de calidad
    quality_stats = analyze_occupancy_quality(horarios_salas, salas_input)
    
    # 3. Crear visualización
    create_quality_chart(quality_stats)
    
    # 4. Guardar resultados
    if save_quality_results(quality_stats):
        print("\nAnálisis de calidad de ocupación completado exitosamente")
    else:
        print("\nError al completar el análisis de calidad de ocupación")
    
    # 2. Crear DataFrame con estadísticas y guardar CSV
    df_stats = create_summary_dataframe(horarios_salas)
    if df_stats is None:
        return
    
    # 3. Calcular estadísticas diarias
    daily_stats = calculate_daily_stats(horarios_salas)
    
    # 4. Crear los tres gráficos
    success = True
    
    # 4.1 Gráfico de ocupación general y obtener totales
    total_ocupados = df_stats['Bloques Ocupados'].sum()
    total_desocupados = df_stats['Bloques Desocupados'].sum()
    total_counts = [total_ocupados, total_desocupados]
    
    if not general_occupancy(df_stats):
        success = False
    
    # 4.2 Gráfico de ocupación por sala
    if not room_occupancy(df_stats):
        success = False
    
    # 4.3 Gráfico de ocupación por día
    if not day_occupancy(horarios_salas):
        success = False

    if not analyze_occupancy_efficiency(horarios_salas, salas_input):
        success = False
    
    # 5. Guardar resultados en JSON
    if not save_results(df_stats, total_counts, daily_stats):
        success = False
    
    if success:
        print("\nAnálisis de ocupación completado exitosamente")
    else:
        print("\nError al completar el análisis de ocupación")

if __name__ == "__main__":
    main()


Análisis de calidad de ocupación completado exitosamente


NameError: name 'analyze_occupancy_efficiency' is not defined